In [ ]:
# 입력 데이터 형식 예시
pdf_data = [
    {
        "page": int,              # 페이지 번호 (1부터 시작)
        "text": str,              # 페이지에서 추출된 전체 텍스트
        "images": [               # 해당 페이지에서 추출된 모든 이미지 정보 리스트
            {
                "x0": float,      # 이미지 좌측 상단 x좌표
                "y0": float,      # 이미지 좌측 상단 y좌표
                "x1": float,      # 이미지 우측 하단 x좌표
                "y1": float,      # 이미지 우측 하단 y좌표
                "image": PIL.Image.Image,   # 이미지 객체 (Pillow)
                "image_type": str           # GPT-4o가 분류한 타입 ("그래프/도표", "로고", "그외")
            },
            ...
        ]
    },
]


In [ ]:

from typing import List, Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_core.documents import Document

# 발표 대본 생성 시스템에서 상태를 관리하는 데이터 구조
class ScriptGenState(TypedDict):
    '''발표 대본 생성 시스템에서 상태를 관리하는 데이터 구조'''
    input_text: str  # PPT에서 추출된 텍스트 (현재 페이지 내용)
    input_document = str # PPT에 관한 전체 문서서
    previous_script: Annotated[List[str], add_messages]  # 이전 페이지의 발표 대본을 저장하여 문맥을 유지
    reasoning: str  # 발표 대본을 생성하는 과정에서 모델이 고려한 사항 (예: 요약, 강조할 부분 등)
    documents: List[Document]  # PPT에서 추출된 추가 정보 (OCR된 이미지 텍스트, 표, 캡션 등)
    generated_script: str  # 최종 생성된 발표 대본 (현재 페이지에 대한 발표 내용)
    mode: str  # 시스템의 동작 모드 (예: "text_only", "image_included", "structured")


In [1]:
from dotenv import load_dotenv
load_dotenv("../../.env")

True

In [ ]:
from langchain_openai import ChatOpenAI
from typing import Optional

def image_area_ratio_exceeds_threshold(image_size: tuple, page_size: tuple, threshold: float = 0.5):
    """
    이미지 크기와 페이지 크기를 비교하여 특정 임계치를 초과하는지 확인하는 메서드
    :param image_size: (width, height) 형태의 이미지 크기
    :param page_size: (width, height) 형태의 페이지 크기
    :param threshold: 임계값 (기본값 0.5)
    :return: 이미지 영역 비율이 임계값을 초과하면 True, 아니면 False
    """
    image_area = image_size[0] * image_size[1]
    page_area = page_size[0] * page_size[1]
    
    if page_area == 0:
        print('페이지 오류')
    
    image_ratio = image_area / page_area
    return image_ratio >= threshold

def analyze_image_content(image_path: str, input_document: str, previous_script: list, use_ocr: bool = False):
    """
    GPT-4V를 사용하여 이미지 내용을 해석하는 메서드.
    
    :param image_path: 해석할 이미지 파일 경로
    :param input_document: PPT 관련 전체 문서 (문맥 제공)
    :param previous_script: 이전 페이지의 발표 대본 (연속성 고려)
    :param use_ocr: OCR 결과를 함께 보낼지 여부 (기본값 False)
    :return: GPT-4V의 이미지 분석 결과
    """
    # 이미지 base64 인코딩
    base64_image = encode_image_to_base64(image_path)

    # 선택적으로 OCR 수행
    ocr_text = extract_text_from_image(image_path) if use_ocr else None

    # GPT-4V 프롬프트 구성
    prompt = f"""
    다음은 발표 자료에 대한 정보입니다:
    - PPT 관련 문서: {input_document}
    - 이전 발표 대본: {previous_script}
    
    아래 이미지를 분석하고, 발표자가 설명해야 할 내용을 정리해 주세요.
    """

    if use_ocr and ocr_text:
        prompt += f"\n추출된 이미지 내 텍스트:\n{ocr_text}"

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-vision-preview",
            messages=[
                {"role": "system", "content": "당신은 발표 자료에서 이미지를 분석하여 발표자가 쉽게 설명할 수 있도록 돕는 AI입니다."},
                {"role": "user", "content": prompt, "image": base64_image}
            ]
        )

        return response["choices"][0]["message"]["content"].strip()

    except openai.error.OpenAIError as e:
        return f"OpenAI API 호출 오류: {str(e)}"

def generate_presentation_script(input_text: str, input_document: str, previous_script: List[str], use_image: bool):
    """
    GPT-4를 사용하여 페이지별 발표 대본을 생성하는 메서드
    :param input_text: 현재 페이지에서 추출한 텍스트
    :param input_document: PPT에 관한 전체 문서
    :param previous_script: 이전 페이지의 발표 대본
    :param use_image: 이미지 사용 여부
    :return: 생성된 발표 대본 문자열
    """
    prompt = f"""
    다음은 발표 자료에 대한 정보입니다:
    - 현재 페이지 텍스트: {input_text}
    - 전체 문서 정보: {input_document}
    - 이전 발표 대본: {previous_script}
    - 이미지 사용 여부: {'Yes' if use_image else 'No'}
    
    위 정보를 바탕으로 발표자가 자연스럽게 발표할 수 있도록 대본을 작성해주세요.
    발표 대본은 논리적인 흐름을 유지하고, 강조해야 할 핵심 내용을 포함해야 합니다.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 발표 대본을 작성하는 AI입니다."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response["choices"][0]["message"]["content"].strip()

In [ ]:
def filter_images(pdf_data):
    """
    이미지 타입에 따라 사용할 이미지를 필터링하는 함수
    Args:
        pdf_data (list): PDF에서 추출한 페이지별 데이터
        
    Returns:
        list: 필터링된 이미지 정보가 포함된 페이지별 데이터
    """
    filtered_data = []
    
    for page in pdf_data:
        filtered_images = []
        for img in page["images"]:
            # 그래프/도표만 사용하고 로고와 기타 이미지는 제외
            if img["image_type"] == "그래프/도표":
                filtered_images.append(img)
                
        filtered_page = {
            "page": page["page"],
            "text": page["text"],
            "images": filtered_images
        }
        filtered_data.append(filtered_page)
    
    return filtered_data

In [ ]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

def generate_script(full_ppt_data, page_text, images, model="gpt-4o-mini"):
    """
    PPT 페이지의 텍스트와 이미지 정보를 바탕으로 발표 대본을 생성하는 함수
    
    Args:
        full_ppt_data (list): 전체 PPT 데이터
        page_text (str): 현재 페이지의 텍스트
        images (list): 현재 페이지의 이미지 정보 리스트
        model (str): 사용할 LLM 모델명, 기본값은 mistral:7b
        
    Returns:
        str: 생성된 발표 대본
    """
    
    # Chatollama 인스턴스 생성
    llm = ChatOpenAI(model=model, temperature=0.5)
    
    # 이미지 설명 텍스트 생성
    image_descriptions = []
    for img in images:
        if img["image_type"] == "그래프/도표":
            desc = f"페이지에 {img['x0']:.1f}, {img['y0']:.1f} 위치에 그래프/도표가 있습니다."
            image_descriptions.append(desc)
    
    # 프롬프트 구성
    prompt = f"""
다음은 발표 자료의 한 페이지 내용입니다. 이를 바탕으로 자연스러운 발표 대본을 작성해주세요.

페이지 텍스트:
{page_text}

시각자료 정보:
{' '.join(image_descriptions)}

전체 발표 맥락을 고려하여, 청중이 이해하기 쉽게 설명하는 2-3문단 분량의 대본을 작성해주세요.
"""

    # LLM을 통한 대본 생성
    response = llm.generate(prompt)
    
    return response.strip()
